# Timed points <img align="right" width="192" height="64" src="https://freemelt.com/app/uploads/freemeltLogo-1.png">
Up until this point, line melting has been the basis of all guides. However, `obplib` has the capability of spot melting as well. There are multiple reasons for spot melting, many of which are summarized here (LINK).

In [1]:
import obplib as obp
import interactive_viewer as iw

#libraries for this guide
import random as rnd
import os

## The `TimedPoints` object
The `obp.TimedPoints` object has three attributes:
1. `points` is a list of `Point` objects. These contain the coordinates of each spot to be melted.
2. `dwellTimes` is a list of the same length as `points`. These denote the time (integer) that the beam stays on each spot, meaning how much energy is deposited into the powder at that location. These can either be individually set to correspond to each `Point` object:
if `object.points = [point_1, point_2, point_3]` then `object.dwellTimes = [dwell_time_1, dwell_time_2, dwell_time_3]`. Alternatively, if all `Point`'s should have the same `dwell_time`, then `object.dwellTimes = [dwell_time_constant, 0, 0]`.
3. `bp` is the same `obp.Beamparameters` object that all other `obplib` objects share. 

### Example - Four points
In this example, four `Point` objects are initialized in a 2 $\times$ 2 grid. Corresponding dwell times are created and a `TimedPoints` object is created, then plotted.

In [2]:
points = [obp.Point(i, j) for i in range(2) for j in range(2)] #initialize points
dwell_times = [100 * (i + 1) for i in range(len(points))]
bp = obp.Beamparameters(1000, 100)
obj = [obp.TimedPoints(points, dwell_times, bp)]
iw.interactive_viewer(obj)


interactive(children=(IntSlider(value=2, description='upper_lim', max=4, min=1), IntText(value=4, description=…

If constant dwell times for a number of points are desired, the dwell_time list can conveniently be defined as

`dwell_time = [dwell_time_const, 0, 0, ..., 0]`, like so:

In [3]:
points_2 = points #reuse points from previous example
#define dwell times
dwell_time_const = 500
dwell_times_2 = [dwell_time_const, 0, 0, 0]
#or, using list comprehension (more flexible)
dwell_times_2 = [dwell_time_const if i == 0 else 0 for i in range(len(points))]
obj_2 =[obp.TimedPoints(points_2, dwell_times_2, bp)] #reuse BeamParameters from previous example
iw.interactive_viewer(obj_2)

interactive(children=(IntSlider(value=2, description='upper_lim', max=4, min=1), IntText(value=4, description=…

## Randomized Spot Melting
Randomizing the melting pattern of a material is an effective way to homogenize it and prevent the formation of internal structures caused by repetitive melting. For this purpose, spot melting is a commonly employed technique, and when it comes to random melting strategies, spot melting is particularly suitable due to its straightforward implementation. Freemelt's Pixelmelt service offers a self-contained solution to achieve random spot melting for materials of any shape.

### Example - Random Spot Melting of a Rectangle
Let's consider a simple example of random spot melting applied to a rectangle. The first step is to define the rectangle and then divide it into a grid of `Point` objects. Subsequently, the list containing these points is shuffled randomly, and a TimedPoints object is initialized, where each point has an associated random dwell time. The results are then visualized.


In [4]:
filename = 'random_timed_points.obp'

try:    
    os.remove(filename)
except:
    pass

rectangle_dimensions = [200, 100]
rectangle_midpoint = [0, 0]
point_list = []

point_density = 0.1 #number of points per micrometer
number_of_points = [int(rectangle_dimensions[0] * point_density), int(rectangle_dimensions[1] * point_density)] #number of points in each dimension

#populate point_list
for i in range(number_of_points[0]):
    for j in range(number_of_points[1]):
        x = rectangle_midpoint[0] - rectangle_dimensions[0] / 2 + i * rectangle_dimensions[0] / number_of_points[0]
        y = rectangle_midpoint[1] - rectangle_dimensions[1] / 2 + j * rectangle_dimensions[1] / number_of_points[1]
        point_list.append(obp.Point(x, y))

rnd.shuffle(point_list) #shuffles the point list

dwell_times  = [int(50 + rnd.random() * 100) for _ in range(len(point_list))] #uniformly random dwell times between 50 and 150 microseconds
#dwell_time_const = 100
#dwell_times = [dwell_time_const if i == 0 else 0 for i in range(len(point_list))] #constant dwell time, uncomment to test
bp = obp.Beamparameters(1000, 100) #create beam parameters

obj = [obp.TimedPoints(point_list, dwell_times, bp)]
obp.write_obp(obj, filename) #save .obp file

iw.interactive_viewer(obj)


interactive(children=(IntSlider(value=100, description='upper_lim', max=200, min=1), IntText(value=200, descri…